In [1]:
import os
%pwd

'c:\\Users\\LONG\\Computer Science\\Artificial Intelligence\\Projects\\End-To-End-Chicken-Disease-Classification\\research'

In [2]:
os.chdir('../')
%pwd

'c:\\Users\\LONG\\Computer Science\\Artificial Intelligence\\Projects\\End-To-End-Chicken-Disease-Classification'

In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        training_data = os.path.join(
            self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"
        )
        eval_config = EvaluationConfig(
            path_of_model=Path(self.config.training.trained_model_path),
            training_data=Path(training_data),
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        data_generator_kwargs = dict(rescale=1.0 / 255, validation_split=0.3)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )

        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("score.json"), data=scores)

In [15]:
try:
    config = ConfigurationManager()
    valid_config = config.get_validation_config()
    evaluation = Evaluation(valid_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e

[2025-05-15 21:24:20,896: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-15 21:24:20,910: INFO: common: yaml file: params.yaml loaded successfully]


[2025-05-15 21:24:20,917: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 2s 198ms/step - loss: 5.5930 - accuracy: 0.8103
[2025-05-15 21:24:24,377: INFO: common: json file saved at: score.json]
